In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas
import datetime as dt

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


### Some Extra Processing

In [2]:
df = pd.read_csv('nba_processed.csv')

In [3]:
df.head()

,Date,League,Entry,Type,Investment,Odds,Gain,Result
0,2022-10-18,NBA,Embiid DD + Harris ab. 19 pts,Pair,10,1.71,7.1,Green
1,2022-10-18,NBA,"Brown 1ºqt ac. 5,5 pts",Points,10,1.80,8.0,Red
2,2022-10-18,NBA,"Celtics ac. 55,5 pts 1T",Total,10,1.86,8.6,Green
3,2022-10-18,NBA,"Maxey 15 pts + White ac. 6,5 pts",Pair,10,1.57,5.7,Red
4,2022-10-18,NBA,"GSW ac. 57,5 pts",Total,10,1.90,9.0,Green


In [4]:
df['Gain'] = np.where(df.Result == 'Red', df.Gain*-1, df.Gain)

In [5]:
df.head()

,Date,League,Entry,Type,Investment,Odds,Gain,Result
0,2022-10-18,NBA,Embiid DD + Harris ab. 19 pts,Pair,10,1.71,7.1,Green
1,2022-10-18,NBA,"Brown 1ºqt ac. 5,5 pts",Points,10,1.80,-8.0,Red
2,2022-10-18,NBA,"Celtics ac. 55,5 pts 1T",Total,10,1.86,8.6,Green
3,2022-10-18,NBA,"Maxey 15 pts + White ac. 6,5 pts",Pair,10,1.57,-5.7,Red
4,2022-10-18,NBA,"GSW ac. 57,5 pts",Total,10,1.90,9.0,Green


In [6]:
df['Profit'] = df.Gain.cumsum()

In [7]:
ls = list()
j = 0
for i in range(len(df)):
    if df.iloc[i].Result == 'Green':
        j += 1
    ls.append(j/(i+1))

In [8]:
df['Win%'] = ls

In [9]:
df.head()

,Date,League,Entry,Type,Investment,Odds,Gain,Result,Profit,Win%
0,2022-10-18,NBA,Embiid DD + Harris ab. 19 pts,Pair,10,1.71,7.1,Green,7.1,1.000000
1,2022-10-18,NBA,"Brown 1ºqt ac. 5,5 pts",Points,10,1.80,-8.0,Red,-0.9,0.500000
2,2022-10-18,NBA,"Celtics ac. 55,5 pts 1T",Total,10,1.86,8.6,Green,7.7,0.666667
3,2022-10-18,NBA,"Maxey 15 pts + White ac. 6,5 pts",Pair,10,1.57,-5.7,Red,2.0,0.500000
4,2022-10-18,NBA,"GSW ac. 57,5 pts",Total,10,1.90,9.0,Green,11.0,0.600000


In [10]:
df.Date = pd.to_datetime(df.Date).dt.date

In [11]:
idf = df.interactive()

In [12]:
df.dtypes

Date           object
League         object
Entry          object
Type           object
Investment      int64
Odds          float64
Gain          float64
Result         object
Profit        float64
Win%          float64
dtype: object

### Rolling Gain/Win Rate

In [13]:
date_slider = pn.widgets.DateSlider(name='Date Slider', start=df.loc[0].Date,
                                        end=df.loc[len(df)-1].Date, value=df.loc[len(df)-1].Date, width=300, align='start')

date_slider

DateSlider(end=datetime.date(2022, ..., name='Date Slider', start=datetime.date(2022, ..., value=datetime.date(2022, ..., width=300)

In [14]:
rolling_pipeline = (
    idf[
        (idf.Date <= date_slider) 
    ].groupby('Date').sum(numeric_only=True)['Gain']
)

In [15]:
rolling_plot = rolling_pipeline.hvplot.bar(x='Date', y='Gain', title='Over Time', rot=70,
                                          color=(df.groupby('Date').sum(numeric_only=True)['Gain']>0),
                                          cmap=['Green', 'Red'], legend=False)
rolling_plot

### Gain/Win% from Each Type of Bet

In [16]:
cross_select = pn.widgets.CrossSelector(name='Type of Bet', options=list(set(df.Type)), value=list(set(df.Type)), sizing_mode='scale_height', width=300)
cross_select

CrossSelector(name='Type of Bet', options=['Turnovers', ...], sizing_mode='scale_height', value=['Turnovers', ...], width=300)

In [17]:
idf_2 = idf[(idf.Date<=date_slider)][['Type', 'Gain']].groupby('Type').sum().reset_index()
bar_pipeline = (
    idf_2[idf_2.Type.isin(cross_select)].reset_index(drop=True)
)

In [18]:
select_plot = bar_pipeline.hvplot.bar(x='Type', y='Gain', title='Based on Type of Bet', rot=70,
                                       color='Gain', cmap=['red', 'green'])
select_plot

### Table for Wins/Losses from bets on each player

In [19]:
props = ['3 Pointers', 'Assists', 'Ast + Reb', 'Blocks', 'Points', 'Pts + Reb', 'Pts + Reb + Ast',
        'Rebounds', 'Steals', 'Stl + Blk', 'Turnovers']
tab_df = df[df.Type.isin(props)].reset_index(drop=True)

In [20]:
tab_df['Entry'] = tab_df['Entry'].str.split(' ').str[0]
tab_df['Entry'].replace({'BOgdanovic':'Bogdanovic'}, inplace=True)
tab_df.rename(columns={'Entry':'Player'}, inplace=True)
temp = tab_df.copy()
tab_df = tab_df.groupby(['Player', 'Type']).sum(numeric_only=True)
tab_df['W'] = temp.groupby(['Player', 'Type'])['Result'].apply(lambda x:x[x.str.contains('Green')].count())
tab_df['L'] = temp.groupby(['Player', 'Type'])['Result'].apply(lambda x:x[x.str.contains('Red')].count())
tab_df.drop(columns=['Profit', 'Win%', 'Odds'], inplace=True)
tab_df=tab_df.sort_values('Gain', ascending=False).reset_index()

In [21]:
tab_df.head()

,Player,Type,Investment,Gain,W,L
0,Haliburton,Points,70,39.5,6,1
1,Shai,Points,70,39.0,6,1
2,Haliburton,Assists,60,31.6,5,1
3,Bogdanovic,Points,50,25.4,4,1
4,Valanciunas,Points,30,24.9,3,0


In [22]:
search= pn.widgets.TextInput(name='Search Player')

In [23]:
def contains_filter(df, pattern, column):
    if not pattern:
        return df
    return df[df[column].str.contains(pattern)]

In [24]:
table = pn.widgets.Tabulator(tab_df, pagination='remote', page_size=10, show_index=False, layout='fit_data')

In [25]:
table.add_filter(pn.bind(contains_filter, pattern=search, column='Player')) 
player_table = pn.Column('####Player Props', search, table)
player_table

Column
    [0] Markdown(str)
    [1] TextInput(name='Search Player')
    [2] Tabulator(layout='fit_data', page_size=10, pagination='remote', show_index=False, value=          Player  ...)

### Table for Wins/Losses from bets on teams

In [26]:
teamprops = ['Do Not Score', 'Handicap', 'ML', 'Total']
team_df_1 = df[(df.Type.isin(teamprops)) & (~df.Entry.str.contains(' x '))].reset_index(drop=True)
team_df_1 = team_df_1[['Entry', 'Type', 'Investment', 'Gain', 'Result']]
team_df_1['Entry'] = team_df_1['Entry'].str.split(' ').str[0]
team_df_1.head()

,Entry,Type,Investment,Gain,Result
0,Celtics,Total,10,8.6,Green
1,GSW,Total,10,9.0,Green
2,Heat,Handicap,10,-7.1,Red
3,Nets,Total,10,-5.7,Red
4,Grizzlies,ML,10,5.5,Green


In [27]:
team_df_2 = df[(df.Type=='Total') & (df.Entry.str.contains(' x '))].reset_index(drop=True)
team_df_2 = team_df_2[['Entry', 'Type', 'Investment', 'Gain', 'Result']]
temp1 = team_df_2.copy()
temp1['Entry'] = temp1.Entry.str.split(' ').str[0]

temp2 = team_df_2.copy()
temp2['Entry'] = temp2.Entry.str.split(' ').str[2]

In [28]:
team_df_2 = pd.concat([temp1, temp2]).reset_index(drop=True)
team_df_2.head()

,Entry,Type,Investment,Gain,Result
0,Mavs,Total,10,-5.0,Red
1,Bucks,Total,10,-6.6,Red
2,Jazz,Total,10,5.5,Green
3,Heat,Total,10,-5.2,Red
4,Heat,Total,10,-6.2,Red


In [29]:
team_df = pd.concat([team_df_1, team_df_2]).reset_index(drop=True)
team_df.head()

,Entry,Type,Investment,Gain,Result
0,Celtics,Total,10,8.6,Green
1,GSW,Total,10,9.0,Green
2,Heat,Handicap,10,-7.1,Red
3,Nets,Total,10,-5.7,Red
4,Grizzlies,ML,10,5.5,Green


In [30]:
team_df.rename(columns={'Entry':'Team'}, inplace=True)
temp3 = team_df.copy()
team_df = team_df.groupby(['Team', 'Type']).sum(numeric_only=True)
team_df['W'] = temp3.groupby(['Team', 'Type'])['Result'].apply(lambda x:x[x.str.contains('Green')].count())
team_df['L'] = temp3.groupby(['Team', 'Type'])['Result'].apply(lambda x:x[x.str.contains('Red')].count())
team_df=team_df.sort_values('Gain', ascending=False).reset_index()

In [31]:
team_df.head()

,Team,Type,Investment,Gain,W,L
0,Rockets,Total,100,28.2,7,3
1,Wizards,Handicap,40,27.1,4,0
2,Bulls,Total,40,25.7,4,0
3,OKC,Total,80,24.5,6,2
4,Pacers,Handicap,60,23.1,5,1


In [32]:
team_search = pn.widgets.TextInput(name='Search Team')

In [33]:
team_tab = pn.widgets.Tabulator(team_df, pagination='remote', page_size=10, show_index=False, layout='fit_data')

In [34]:
team_tab.add_filter(pn.bind(contains_filter, pattern=team_search, column='Team')) 
team_table = pn.Column('####Team Betting', team_search, team_tab)
team_table

Column
    [0] Markdown(str)
    [1] TextInput(name='Search Team')
    [2] Tabulator(layout='fit_data', page_size=10, pagination='remote', show_index=False, value=       Team   ...)

### Best Performance

In [35]:
day_df = df.groupby('Date').sum(numeric_only=True).reset_index()
best_day = day_df.sort_values(by='Gain', ascending=False)
best_date = best_day.iloc[0].Date.strftime('%Y-%m-%d')
best_gain = best_day.iloc[0].Gain

In [36]:
bet_df = df.groupby('Type').sum(numeric_only=True).reset_index().sort_values(by='Gain', ascending=False)
best_type = bet_df.iloc[0].Type
type_gain = bet_df.iloc[0].Gain

In [37]:
best_player = tab_df.iloc[0].Player
player_type = tab_df.iloc[0].Type
player_gain = tab_df.iloc[0].Gain
player_win = tab_df.iloc[0].W
player_loss = tab_df.iloc[0].L

In [38]:
best_team = team_df.iloc[0].Team
team_type = team_df.iloc[0].Type
team_gain = team_df.iloc[0].Gain
team_win = team_df.iloc[0].W
team_loss = team_df.iloc[0].L

## Create Dashboard

In [40]:
template = pn.template.FastListTemplate(
    title='NBA Betting Dashboard',
    sidebar_width=300,
    sidebar=[pn.pane.Markdown('# Welcome to Your Dashboard!'),
             pn.pane.Markdown('Your Best Day is on **{}** with Gain of **{}**.'.format(best_date, best_gain)),
             pn.pane.Markdown('Your Best Bets are from **{}** bets with Gain of **{}**.'.format(best_type, type_gain)),
             pn.pane.Markdown('Your Best Player Prop Bets came from betting on **{} {}** with Gain of **{}** and Record of **{}** Wins and **{}** Losses.'.format(best_player, player_type, player_gain, player_win, player_loss)),
             pn.pane.Markdown('Your Best Team Bets came from betting on **{} {}** with Gain of **{}** and Record of **{}** Wins and **{}** Losses.'.format(best_team, team_type, round(team_gain, 1), team_win, team_loss)),
             pn.pane.Markdown('More Details can be found in the Graphs and Tables.')
            ],
    main=[pn.Column(pn.pane.Markdown('### Gain/Loss'),
                    date_slider,
                    pn.Column(pn.Row(cross_select,
                                    select_plot.panel(width=500, margin=(0,25)))),
                    rolling_plot.panel(width=1000, height=400, align='center')),
          pn.Column(pn.pane.Markdown('### Performance'),
                    pn.Row(player_table, pn.Spacer(width=100), team_table))
         ]
)

# template.show()
template.servable();